In [9]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.preprocessing import LabelEncoder  # 추가된 라이브러리

# 1. 데이터 로드
file_path = 'AKI_cohort.csv'  # 파일 경로 확인
df = pd.read_csv(file_path)

In [10]:
# ID 컬럼과 결과 변수(Target) 분리
identifiers = ['subject_id', 'hadm_id', 'stay_id']
target_col = 'aki_outcome'

# 분석에 사용할 변수들만 선택 (ID 및 Target 제외)
X = df.drop(columns=identifiers + [target_col], errors='ignore')
y = df[target_col]

print(f"전처리 시작: 총 {X.shape[1]}개 변수, {X.shape[0]}명 환자 데이터 로드됨.")

# ==============================================================================
# 2. 범주형 변수 숫자 변환 (Label Encoding)
# Gender, Race 등 문자열 데이터를 숫자로 변환합니다.
# ==============================================================================
object_cols = X.select_dtypes(include=['object']).columns
if len(object_cols) > 0:
    print(f"범주형(문자열) 변수 변환 중: {object_cols.tolist()}")
    le = LabelEncoder()
    for col in object_cols:
        # 결측치가 섞여 있을 경우를 대비해 문자열로 변환 후 인코딩
        X[col] = le.fit_transform(X[col].astype(str))
else:
    print("범주형(문자열) 변수가 없습니다.")

# ==============================================================================
# 3. 결측치 처리 (Missing Value Processing) - 논문 방법론 적용
# ==============================================================================

# 3-1. 결측률 20% 초과 변수 제거 [논문 p.3, Data cleaning 섹션]
# SQL로 이미 최종 변수를 뽑았더라도, 우리 데이터에서 결측이 너무 심하면 제외하는 것이 맞습니다.
missing_threshold = 0.20
missing_series = X.isnull().mean()
drop_columns = missing_series[missing_series > missing_threshold].index.tolist()

if len(drop_columns) > 0:
    X_filtered = X.drop(columns=drop_columns)
    print(f"⚠️ 결측률 20% 초과로 제거된 변수 ({len(drop_columns)}개): {drop_columns}")
else:
    X_filtered = X
    print("결측률 20%를 초과하는 변수가 없습니다. (모두 유지)")

print(f"남은 변수 개수: {X_filtered.shape[1]}개")

# 3-2. 다중 대체 (MICE) 적용 [논문 p.3, Data cleaning 섹션]
# 결측값을 다른 변수들과의 상관관계를 이용해 채워 넣습니다.
print("결측치 대체(MICE) 진행 중... (데이터 크기에 따라 시간이 소요될 수 있습니다)")
imputer = IterativeImputer(max_iter=10, random_state=42)
X_imputed_array = imputer.fit_transform(X_filtered)

# 컬럼명과 인덱스 복원
X_final = pd.DataFrame(X_imputed_array, columns=X_filtered.columns, index=X_filtered.index)
print("결측치 대체 완료.")

# ==============================================================================
# 4. 최종 데이터셋 저장
# RFE 단계 생략: SQL에서 추출한 변수 목록을 최종 변수로 확정
# ==============================================================================

# ID, Target, 전처리된 Features를 합쳐서 저장
final_df = pd.concat([df[identifiers], df[[target_col]], X_final], axis=1)

output_filename = 'Processed_MIMIC_Data.csv'
final_df.to_csv(output_filename, index=False)

print("="*50)
print(f"전처리 완료! 파일 저장됨: {output_filename}")
print(f"최종 데이터 크기: {final_df.shape}")
print("="*50)

전처리 시작: 총 51개 변수, 811명 환자 데이터 로드됨.
범주형(문자열) 변수 변환 중: ['gender', 'race', 'insurance', 'admission_type']
결측률 20%를 초과하는 변수가 없습니다. (모두 유지)
남은 변수 개수: 51개
결측치 대체(MICE) 진행 중... (데이터 크기에 따라 시간이 소요될 수 있습니다)
결측치 대체 완료.
전처리 완료! 파일 저장됨: Processed_MIMIC_Data.csv
최종 데이터 크기: (811, 55)


c:\Users\junsy\miniconda3\envs\aki-project\Lib\site-packages\sklearn\impute\_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
